<a href="https://colab.research.google.com/github/dmatrix/LearningSparkV2/blob/master/mlflow_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install mlflow --quiet

In [2]:
import os
import shutil
import pprint

from random import random, randint
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifacts
from sklearn.ensemble import RandomForestRegressor
from mlflow.tracking import MlflowClient
import warnings

In [9]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

1.10.0


In [10]:
# Set the tracking server to be localhost with sqlite as tracking store
local_registry = "sqlite:///mlruns.db"
print(f"Running local model registry={local_registry}")
model_name="WeatherForecastModel"
mlflow.set_tracking_uri(local_registry)
with mlflow.start_run(run_name="LOCAL_REGISTRY") as run:
    params = {"n_estimators": 3, "random_state": 0}
    sk_learn_rfr = RandomForestRegressor(params)

    # Log parameters and metrics using the MLflow API
    mlflow.log_params(params)
    log_param("param_1", randint(0, 100))
    log_metric("metric_1", random())
    log_metric("metric_2", random() + 1)
    log_metric("metric_33", random() + 2)

    # Log and register the model at the same time
    mlflow.sklearn.log_model(
                sk_model = sk_learn_rfr,
                artifact_path = "sklearn-model",
                registered_model_name="WeatherForecastModel")
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("Looks, like I logged to the local store!")
    log_artifacts("outputs")
    shutil.rmtree('outputs')
    run_id = run.info.run_uuid

Running local model registry=sqlite:///mlruns.db


Registered model 'WeatherForecastModel' already exists. Creating a new version of this model...
Created version '3' of model 'WeatherForecastModel'.


In [11]:
client = MlflowClient()
#
# transition model stage to production
#
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="production")

# Get a list of all registered models
print("List of all registered models")
print("=" * 80)
[print(pprint.pprint(dict(rm), indent=4)) for rm in client.list_registered_models()]

# Get a list of specific versions of the named models
print(f"List of Model = {model_name} and Versions")
print("=" * 80)
[pprint.pprint(dict(mv), indent=4) for mv in client.search_model_versions("name='WeatherForecastModel'")]


List of all registered models
{   'creation_timestamp': 1598045102538,
    'description': None,
    'last_updated_timestamp': 1598045373358,
    'latest_versions': [   <ModelVersion: creation_timestamp=1598045102589, current_stage='Production', description=None, last_updated_timestamp=1598045373358, name='WeatherForecastModel', run_id='14e42a7c16fe49298afbd5ba3f346c44', source='./mlruns/0/14e42a7c16fe49298afbd5ba3f346c44/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=1>,
                           <ModelVersion: creation_timestamp=1598045363554, current_stage='None', description=None, last_updated_timestamp=1598045363554, name='WeatherForecastModel', run_id='d0a6ddcc135c493da1b49a7b40f1bafa', source='./mlruns/0/d0a6ddcc135c493da1b49a7b40f1bafa/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=3>],
    'name': 'WeatherForecastModel',
    'tags': {}}
None
List of Model = WeatherForecastModel and Ve

[None, None, None]

In [12]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000 &")# run tracking UI in the background

In [13]:
# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6
!pip install pyngrok --quiet
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
public_url = ngrok.connect(port="5000", proto="http", options={"bind_tls": True})
print("MLflow Tracking UI:", public_url)

MLflow Tracking UI: https://ec4fbf689552.ngrok.io
